# Code to generate a series of random structures with varying stoichiometries, suitable for RSS.

Examples shown here primarily focus on the elements, C, H and O, but the code readily generalises to any elements.

In [5]:
from ase import Atom, Atoms, io
import numpy as np
import nglview as nv
import os
import sys

In [6]:
class boxbuilder:
    
    def __init__(self, dens, stoi, rmin=2.0, name='rss_cell'):
        
        self.stoi = stoi  # a dictionary indicating the number of each element
        if type(self.stoi) == str:
            self.stoi = self.stoi_string_to_dict(self.stoi)
        
        self.dens = dens # a float indicating the target density of the system of interest
        
        self.rmin = rmin
        self.name = name
        self.cell_guess = None
        
        self.mass = self.box_mass_from_stoi()
        self.box_l = self.compute_box_lengths()
        self.atoms_obj = self.gen_empty_box()
        self.populate_box()
        
    def stoi_string_to_dict(self, stoi_string):
        """If the function is passed a string rather than a dictionary, convert it"""
        
        at_temp = Atoms(stoi_string)
        elem = []
        stoi_dict = {}

        # get the unique elements in the cell
        for atom in at_temp:
            if atom.symbol not in elem:
                elem.append(atom.symbol)

        # initialise the dictionary
        for element in elem:
            stoi_dict[element] = 0

        # count atoms of each element into dictionary
        for atom in at_temp:
            stoi_dict[atom.symbol] += 1

        return stoi_dict
        
    def gen_empty_box(self):
        """Generate an empty box for us to put atoms into"""
        
        self.cell_guess = [self.box_l, self.box_l, self.box_l]
        at = Atoms('', positions=[], pbc=True, cell=self.cell_guess)
        return at
        
    def box_mass_from_stoi(self):
        """Computes the mass of the desired cell from the stoichiometry given"""
        
        mass = 0
        
        for elem, num in self.stoi.items():
            atom = Atom(elem, [0, 0, 0])
            mass += atom.mass * num
             
        print(r"Total Box Mass: " + str(mass) + ' g')
        return mass
    
    def compute_box_lengths(self):
        """Compute the box length, assuming a cube, from the mass & density"""
        
        length = (self.mass / self.dens) ** (1/3)
        
        print("Computed ideal box length for dens %f is %f Angstrom" % (self.dens, length))
        
        return length
        
    def populate_box(self):
        mindist_init = self.rmin
        mindist = mindist_init
        nfailed = 1
        
        n_at = 0
        
        for elem, num_el in self.stoi.items():
            n_el = 0
            while n_el < num_el:
                at = Atoms(elem, positions=[[0, 0, 0]], cell=self.cell_guess)

                at.positions[:, 0] += np.random.rand() * self.atoms_obj.get_cell()[0,0]
                at.positions[:, 1] += np.random.rand() * self.atoms_obj.get_cell()[1,1]
                at.positions[:, 2] += np.random.rand() * self.atoms_obj.get_cell()[2,2]

                self.atoms_obj.extend(at)
                
                if n_at == 0:
                    n_at += 1
                    n_el += 1
                    continue
                
                dists = self.atoms_obj.get_distances(-1, np.arange(0, len(self.atoms_obj)-1), mic=True)
                
                if np.array(dists).min() > mindist:
                    #mindist = mindist_init
                    sys.stdout.write('\r')
                    sys.stdout.flush()
                    sys.stdout.write("%s: %i / %i : Minimum distance = %f" % (elem, n_el + 1, num_el, np.array(dists).min()))
                    n_at += 1
                    n_el += 1
                else:
                    del self.atoms_obj[-1]
                    nfailed += 1

                if nfailed % 5000 == 0:
                    mindist -= 0.01
                    print("\n Setting new minimum interatomic distance after many failed placements: " + str(mindist))
                    
    def view_structure(self):
        return nv.show_ase(self.atoms_obj)
    
    def save_struct(self, save_path='./', stoi_in_name=False):
        
        stoi_string = ''
        for elem, n_elem in self.stoi.items():
            stoi_string += str(elem) + str(n_elem)
        
        if stoi_in_name:
            name_full = self.name + '_' + stoi_string
        else:
            name_full = self.name
        
        # Always ends up being useful to just save in a few file formats right from the start
        io.write(os.path.join(save_path, name_full + '.POSCAR'), self.atoms_obj, format='vasp', sort=True)
        io.write(os.path.join(save_path, name_full + '.pdb'), self.atoms_obj, format='proteindatabank')
        io.write(os.path.join(save_path, name_full + '.xyz'), self.atoms_obj, format='extxyz')
        
        # And to save the cell for CP2K
        # a, b, c, al, bet, gam = self.atoms_obj.get_cell_lengths_and_angles()
        # cell = np.array([a, b, c])
        header = np.array([['A', 'B', 'C']])
        cp2k_cell = np.hstack((header.T, self.atoms_obj.cell))
        np.savetxt(os.path.join(save_path, name_full + '.cell'), cp2k_cell, fmt="%s")

# A quick test example to build a single structure

In [10]:
# Stoichiometry input can be either in the form of a simple dictionary
test = boxbuilder(1.0, {'C':20, 'H':5, 'O':5})

# Or as a string specifying the chemical formula desired.
test_2 = boxbuilder(1.0, 'C20H5O5')

Total Box Mass: 325.255 g
Computed ideal box length for dens 1.000000 is 6.877142 Angstrom
O: 1 / 5 : Minimum distance = 2.05137313
 Setting new minimum interatomic distance after many failed placements: 1.99
O: 2 / 5 : Minimum distance = 2.000244
 Setting new minimum interatomic distance after many failed placements: 1.98
O: 3 / 5 : Minimum distance = 1.999651
 Setting new minimum interatomic distance after many failed placements: 1.97
O: 5 / 5 : Minimum distance = 1.980197Total Box Mass: 325.255 g
Computed ideal box length for dens 1.000000 is 6.877142 Angstrom
H: 5 / 5 : Minimum distance = 2.03273113
 Setting new minimum interatomic distance after many failed placements: 1.99

 Setting new minimum interatomic distance after many failed placements: 1.98
O: 2 / 5 : Minimum distance = 1.999944
 Setting new minimum interatomic distance after many failed placements: 1.97

 Setting new minimum interatomic distance after many failed placements: 1.96

 Setting new minimum interatomic distan

In [11]:
nv.show_ase(test.atoms_obj)

NGLWidget()

In [5]:
nv.show_ase(test_2.atoms_obj)

# Bigger 'production run' example. To build several hundred structures with varying stoichiometries.

In [3]:
def build_stoichiometries():
    """Quick hardcoded helper function to loop over many random stoichiometries"""
    
    
    stoichiometries_other = []
    stoichiometries_h2o = []

    for n_o in np.arange(0,8,1):
        for n_h in np.arange(0,16,2):
            stoi_string = "C100"

            if int(n_o) != 0:
                stoi_string += "O" + str(int(n_o))

            if int(n_h) != 0:
                stoi_string += "H" + str(int(n_h))

            # stoichiometries.append(stoi_string)

            if int(n_o) != 0:
                if int(n_h)/int(n_o) != 2:
                    stoichiometries_other.append(stoi_string)

    for n_o in np.arange(0,52,8):
        for n_h in np.arange(0,52,8):
            stoi_string = "C100"

            if int(n_o) != 0:
                stoi_string += "O" + str(int(n_o))

            if int(n_h) != 0:
                stoi_string += "H" + str(int(n_h))

            # stoichiometries.append(stoi_string)

            if int(n_o) != 0:
                if int(n_h)/int(n_o) != 2:
                    stoichiometries_other.append(stoi_string)

    for n_o in np.arange(0,52,1):
        for n_h in np.arange(0,52,1):
            stoi_string = "C100"

            if int(n_o) != 0:
                stoi_string += "O" + str(int(n_o))

            if int(n_h) != 0:
                stoi_string += "H" + str(int(n_h))

            # stoichiometries.append(stoi_string)

            if int(n_o) != 0:
                if int(n_h)/int(n_o) == 2:
                    stoichiometries_h2o.append(stoi_string)

    return stoichiometries_other, stoichiometries_h2o

In [4]:
densities = [1.5, 2.0, 2.5]

stoichiometries_other, stoichiometries_h2o = build_stoichiometries()

NameError: name 'np' is not defined

In [9]:
if not os.path.exists('./st_other'):
    os.mkdir('./st_other')

for dens in densities:
    if not os.path.exists(os.path.join('./st_other', str(dens))):
        os.mkdir(os.path.join('./st_other', str(dens)))
    for stoi in stoichiometries_other:
        if not os.path.exists(os.path.join('./st_other', str(dens), stoi)):
            os.mkdir(os.path.join('./st_other', str(dens), stoi))
        builder = boxbuilder(dens, stoi)
        builder.save_struct(save_path=os.path.join('./st_other', str(dens), stoi))

Total Box Mass: 1217.099 g
Computed ideal box length for dens 1.500000 is 9.327062 Angstrom
C: 62 / 100 : Minimum distance = 2.023540
 Setting new minimum interatomic distance after many failed placements: 1.99
C: 64 / 100 : Minimum distance = 2.014975
 Setting new minimum interatomic distance after many failed placements: 1.98

 Setting new minimum interatomic distance after many failed placements: 1.97
C: 65 / 100 : Minimum distance = 1.996885
 Setting new minimum interatomic distance after many failed placements: 1.96
C: 68 / 100 : Minimum distance = 2.004002
 Setting new minimum interatomic distance after many failed placements: 1.95

 Setting new minimum interatomic distance after many failed placements: 1.94

 Setting new minimum interatomic distance after many failed placements: 1.93
C: 70 / 100 : Minimum distance = 1.941527
 Setting new minimum interatomic distance after many failed placements: 1.92

 Setting new minimum interatomic distance after many failed placements: 1.91
C

H: 4 / 8 : Minimum distance = 1.638843
 Setting new minimum interatomic distance after many failed placements: 1.6199999999999997
H: 8 / 8 : Minimum distance = 1.631475Total Box Mass: 1227.1789999999999 g
Computed ideal box length for dens 1.500000 is 9.352741 Angstrom
C: 64 / 100 : Minimum distance = 2.007662
 Setting new minimum interatomic distance after many failed placements: 1.99
C: 66 / 100 : Minimum distance = 2.072823
 Setting new minimum interatomic distance after many failed placements: 1.98
C: 67 / 100 : Minimum distance = 2.007271
 Setting new minimum interatomic distance after many failed placements: 1.97
C: 68 / 100 : Minimum distance = 1.973489
 Setting new minimum interatomic distance after many failed placements: 1.96

 Setting new minimum interatomic distance after many failed placements: 1.95
C: 70 / 100 : Minimum distance = 1.971001
 Setting new minimum interatomic distance after many failed placements: 1.94
C: 71 / 100 : Minimum distance = 1.961158
 Setting new mi

O: 1 / 1 : Minimum distance = 1.7085334985
 Setting new minimum interatomic distance after many failed placements: 1.6899999999999997

 Setting new minimum interatomic distance after many failed placements: 1.6799999999999997
H: 1 / 12 : Minimum distance = 1.708372
 Setting new minimum interatomic distance after many failed placements: 1.6699999999999997
H: 3 / 12 : Minimum distance = 1.730889
 Setting new minimum interatomic distance after many failed placements: 1.6599999999999997

 Setting new minimum interatomic distance after many failed placements: 1.6499999999999997
H: 5 / 12 : Minimum distance = 1.672511
 Setting new minimum interatomic distance after many failed placements: 1.6399999999999997

 Setting new minimum interatomic distance after many failed placements: 1.6299999999999997
H: 7 / 12 : Minimum distance = 1.641912
 Setting new minimum interatomic distance after many failed placements: 1.6199999999999997
H: 8 / 12 : Minimum distance = 1.657115
 Setting new minimum inter

C: 86 / 100 : Minimum distance = 1.798732
 Setting new minimum interatomic distance after many failed placements: 1.7699999999999998
C: 89 / 100 : Minimum distance = 1.801826
 Setting new minimum interatomic distance after many failed placements: 1.7599999999999998
C: 91 / 100 : Minimum distance = 1.793145
 Setting new minimum interatomic distance after many failed placements: 1.7499999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7399999999999998
C: 92 / 100 : Minimum distance = 1.759110
 Setting new minimum interatomic distance after many failed placements: 1.7299999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7199999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7099999999999997
C: 94 / 100 : Minimum distance = 1.732297
 Setting new minimum interatomic distance after many failed placements: 1.6999999999999997
C: 95 / 100 : Minimum distance = 1.718512
 Setting new mini

C: 94 / 100 : Minimum distance = 1.787866
 Setting new minimum interatomic distance after many failed placements: 1.7699999999999998
C: 95 / 100 : Minimum distance = 1.776403
 Setting new minimum interatomic distance after many failed placements: 1.7599999999999998
C: 97 / 100 : Minimum distance = 1.762409
 Setting new minimum interatomic distance after many failed placements: 1.7499999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7399999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7299999999999998
C: 98 / 100 : Minimum distance = 1.775734
 Setting new minimum interatomic distance after many failed placements: 1.7199999999999998
O: 1 / 2 : Minimum distance = 1.7487264857
 Setting new minimum interatomic distance after many failed placements: 1.7099999999999997
H: 1 / 10 : Minimum distance = 1.717313
 Setting new minimum interatomic distance after many failed placements: 1.6999999999999997

 Setting new minim

...

C: 60 / 100 : Minimum distance = 2.012118
 Setting new minimum interatomic distance after many failed placements: 1.99
C: 62 / 100 : Minimum distance = 2.015570
 Setting new minimum interatomic distance after many failed placements: 1.98
C: 64 / 100 : Minimum distance = 1.981182
 Setting new minimum interatomic distance after many failed placements: 1.97
C: 65 / 100 : Minimum distance = 1.970570
 Setting new minimum interatomic distance after many failed placements: 1.96

 Setting new minimum interatomic distance after many failed placements: 1.95

 Setting new minimum interatomic distance after many failed placements: 1.94

 Setting new minimum interatomic distance after many failed placements: 1.93
C: 69 / 100 : Minimum distance = 1.939580
 Setting new minimum interatomic distance after many failed placements: 1.92
C: 70 / 100 : Minimum distance = 1.921386
 Setting new minimum interatomic distance after many failed placements: 1.91
C: 75 / 100 : Minimum distance = 1.927301
 Setting n


 Setting new minimum interatomic distance after many failed placements: 1.88
C: 75 / 100 : Minimum distance = 1.897334
 Setting new minimum interatomic distance after many failed placements: 1.8699999999999999

 Setting new minimum interatomic distance after many failed placements: 1.8599999999999999
C: 79 / 100 : Minimum distance = 1.890912
 Setting new minimum interatomic distance after many failed placements: 1.8499999999999999
C: 81 / 100 : Minimum distance = 1.874238
 Setting new minimum interatomic distance after many failed placements: 1.8399999999999999

 Setting new minimum interatomic distance after many failed placements: 1.8299999999999998
C: 82 / 100 : Minimum distance = 1.840490
 Setting new minimum interatomic distance after many failed placements: 1.8199999999999998

 Setting new minimum interatomic distance after many failed placements: 1.8099999999999998
C: 84 / 100 : Minimum distance = 1.816005
 Setting new minimum interatomic distance after many failed placements: 

C: 81 / 100 : Minimum distance = 1.858522
 Setting new minimum interatomic distance after many failed placements: 1.8199999999999998
C: 82 / 100 : Minimum distance = 1.854405
 Setting new minimum interatomic distance after many failed placements: 1.8099999999999998
C: 83 / 100 : Minimum distance = 1.816133
 Setting new minimum interatomic distance after many failed placements: 1.7999999999999998
C: 86 / 100 : Minimum distance = 1.819050
 Setting new minimum interatomic distance after many failed placements: 1.7899999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7799999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7699999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7599999999999998
C: 87 / 100 : Minimum distance = 1.786014
 Setting new minimum interatomic distance after many failed placements: 1.7499999999999998

 Setting new minimum interatomic distance after many faile

C: 83 / 100 : Minimum distance = 1.792307
 Setting new minimum interatomic distance after many failed placements: 1.7699999999999998
C: 84 / 100 : Minimum distance = 1.826034
 Setting new minimum interatomic distance after many failed placements: 1.7599999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7499999999999998
C: 87 / 100 : Minimum distance = 1.789253
 Setting new minimum interatomic distance after many failed placements: 1.7399999999999998
C: 89 / 100 : Minimum distance = 1.772844
 Setting new minimum interatomic distance after many failed placements: 1.7299999999999998
C: 90 / 100 : Minimum distance = 1.732680
 Setting new minimum interatomic distance after many failed placements: 1.7199999999999998
C: 92 / 100 : Minimum distance = 1.808046
 Setting new minimum interatomic distance after many failed placements: 1.7099999999999997
C: 93 / 100 : Minimum distance = 1.733434
 Setting new minimum interatomic distance after many failed placemen

C: 88 / 100 : Minimum distance = 1.794094
 Setting new minimum interatomic distance after many failed placements: 1.7799999999999998
C: 89 / 100 : Minimum distance = 1.789746
 Setting new minimum interatomic distance after many failed placements: 1.7699999999999998
C: 91 / 100 : Minimum distance = 1.798684
 Setting new minimum interatomic distance after many failed placements: 1.7599999999999998
C: 93 / 100 : Minimum distance = 1.818857
 Setting new minimum interatomic distance after many failed placements: 1.7499999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7399999999999998

 Setting new minimum interatomic distance after many failed placements: 1.7299999999999998
C: 94 / 100 : Minimum distance = 1.733008
 Setting new minimum interatomic distance after many failed placements: 1.7199999999999998
C: 95 / 100 : Minimum distance = 1.745925
 Setting new minimum interatomic distance after many failed placements: 1.7099999999999997
C: 97 / 100 : Mini

In [12]:
if not os.path.exists('./st_h2_o1'):
    os.mkdir('./st_h2_o1')

for dens in densities:
    if not os.path.exists(os.path.join('./st_h2_o1', str(dens))):
        os.mkdir(os.path.join('./st_h2_o1', str(dens)))
    for stoi in stoichiometries_h2o:
        if not os.path.exists(os.path.join('./st_h2_o1', str(dens), stoi)):
            os.mkdir(os.path.join('./st_h2_o1', str(dens), stoi))
        builder = boxbuilder(dens, stoi, rmin=1.8)
        builder.save_struct(save_path=os.path.join('./st_h2_o1', str(dens), stoi))

Total Box Mass: 1219.115 g
Computed ideal box length for dens 1.500000 is 9.332209 Angstrom
C: 84 / 100 : Minimum distance = 1.831248
 Setting new minimum interatomic distance after many failed placements: 1.79
C: 90 / 100 : Minimum distance = 1.814115
 Setting new minimum interatomic distance after many failed placements: 1.78
C: 94 / 100 : Minimum distance = 1.793107
 Setting new minimum interatomic distance after many failed placements: 1.77
C: 95 / 100 : Minimum distance = 1.791786
 Setting new minimum interatomic distance after many failed placements: 1.76
C: 96 / 100 : Minimum distance = 1.781820
 Setting new minimum interatomic distance after many failed placements: 1.75
C: 99 / 100 : Minimum distance = 1.798360
 Setting new minimum interatomic distance after many failed placements: 1.74
O: 1 / 1 : Minimum distance = 1.7830154741
 Setting new minimum interatomic distance after many failed placements: 1.73

 Setting new minimum interatomic distance after many failed placements: 1

H: 18 / 18 : Minimum distance = 1.610806Total Box Mass: 1381.25 g
Computed ideal box length for dens 1.500000 is 9.728824 Angstrom
C: 94 / 100 : Minimum distance = 1.867580
 Setting new minimum interatomic distance after many failed placements: 1.79
O: 3 / 10 : Minimum distance = 1.801205190
 Setting new minimum interatomic distance after many failed placements: 1.78

 Setting new minimum interatomic distance after many failed placements: 1.77

 Setting new minimum interatomic distance after many failed placements: 1.76
O: 6 / 10 : Minimum distance = 1.816069
 Setting new minimum interatomic distance after many failed placements: 1.75

 Setting new minimum interatomic distance after many failed placements: 1.74
O: 8 / 10 : Minimum distance = 1.834910
 Setting new minimum interatomic distance after many failed placements: 1.73

 Setting new minimum interatomic distance after many failed placements: 1.72
O: 10 / 10 : Minimum distance = 1.733742
 Setting new minimum interatomic distance a


 Setting new minimum interatomic distance after many failed placements: 1.64
H: 19 / 26 : Minimum distance = 1.640524
 Setting new minimum interatomic distance after many failed placements: 1.63
H: 21 / 26 : Minimum distance = 1.661430
 Setting new minimum interatomic distance after many failed placements: 1.6199999999999999
H: 22 / 26 : Minimum distance = 1.638288
 Setting new minimum interatomic distance after many failed placements: 1.6099999999999999
H: 25 / 26 : Minimum distance = 1.662190
 Setting new minimum interatomic distance after many failed placements: 1.5999999999999999
H: 26 / 26 : Minimum distance = 1.664141Total Box Mass: 1453.31 g
Computed ideal box length for dens 1.500000 is 9.895149 Angstrom
C: 93 / 100 : Minimum distance = 1.839694
 Setting new minimum interatomic distance after many failed placements: 1.79
O: 2 / 14 : Minimum distance = 1.875272923
 Setting new minimum interatomic distance after many failed placements: 1.78
O: 7 / 14 : Minimum distance = 1.82525

H: 30 / 32 : Minimum distance = 1.617985
 Setting new minimum interatomic distance after many failed placements: 1.5699999999999998
H: 32 / 32 : Minimum distance = 1.608331Total Box Mass: 1507.3549999999998 g
Computed ideal box length for dens 1.500000 is 10.016318 Angstrom
O: 3 / 17 : Minimum distance = 1.811605723
 Setting new minimum interatomic distance after many failed placements: 1.79
O: 8 / 17 : Minimum distance = 1.857692
 Setting new minimum interatomic distance after many failed placements: 1.78

 Setting new minimum interatomic distance after many failed placements: 1.77
O: 10 / 17 : Minimum distance = 1.792405
 Setting new minimum interatomic distance after many failed placements: 1.76
O: 13 / 17 : Minimum distance = 1.764569
 Setting new minimum interatomic distance after many failed placements: 1.75
H: 2 / 34 : Minimum distance = 1.7659120
 Setting new minimum interatomic distance after many failed placements: 1.74
H: 3 / 34 : Minimum distance = 1.742562
 Setting new min

H: 17 / 44 : Minimum distance = 1.713777
 Setting new minimum interatomic distance after many failed placements: 1.65
H: 18 / 44 : Minimum distance = 1.676133
 Setting new minimum interatomic distance after many failed placements: 1.64
H: 24 / 44 : Minimum distance = 1.674793
 Setting new minimum interatomic distance after many failed placements: 1.63
H: 26 / 44 : Minimum distance = 1.664102
 Setting new minimum interatomic distance after many failed placements: 1.6199999999999999
H: 30 / 44 : Minimum distance = 1.622450
 Setting new minimum interatomic distance after many failed placements: 1.6099999999999999

 Setting new minimum interatomic distance after many failed placements: 1.5999999999999999
H: 31 / 44 : Minimum distance = 1.643309
 Setting new minimum interatomic distance after many failed placements: 1.5899999999999999
H: 32 / 44 : Minimum distance = 1.592675
 Setting new minimum interatomic distance after many failed placements: 1.5799999999999998
H: 36 / 44 : Minimum dista

...

H: 5 / 34 : Minimum distance = 1.445376
 Setting new minimum interatomic distance after many failed placements: 1.4099999999999997
H: 7 / 34 : Minimum distance = 1.447184
 Setting new minimum interatomic distance after many failed placements: 1.3999999999999997
H: 12 / 34 : Minimum distance = 1.448390
 Setting new minimum interatomic distance after many failed placements: 1.3899999999999997
H: 15 / 34 : Minimum distance = 1.420139
 Setting new minimum interatomic distance after many failed placements: 1.3799999999999997
H: 18 / 34 : Minimum distance = 1.384406
 Setting new minimum interatomic distance after many failed placements: 1.3699999999999997
H: 20 / 34 : Minimum distance = 1.379237
 Setting new minimum interatomic distance after many failed placements: 1.3599999999999997
H: 21 / 34 : Minimum distance = 1.384209
 Setting new minimum interatomic distance after many failed placements: 1.3499999999999996
H: 22 / 34 : Minimum distance = 1.355087
 Setting new minimum interatomic dist

C: 94 / 100 : Minimum distance = 1.576945
 Setting new minimum interatomic distance after many failed placements: 1.5599999999999998
C: 97 / 100 : Minimum distance = 1.568647
 Setting new minimum interatomic distance after many failed placements: 1.5499999999999998
C: 100 / 100 : Minimum distance = 1.556731
 Setting new minimum interatomic distance after many failed placements: 1.5399999999999998

 Setting new minimum interatomic distance after many failed placements: 1.5299999999999998
O: 1 / 20 : Minimum distance = 1.577974
 Setting new minimum interatomic distance after many failed placements: 1.5199999999999998
O: 4 / 20 : Minimum distance = 1.541513
 Setting new minimum interatomic distance after many failed placements: 1.5099999999999998

 Setting new minimum interatomic distance after many failed placements: 1.4999999999999998
O: 8 / 20 : Minimum distance = 1.524476
 Setting new minimum interatomic distance after many failed placements: 1.4899999999999998
O: 10 / 20 : Minimum di

H: 11 / 42 : Minimum distance = 1.465589
 Setting new minimum interatomic distance after many failed placements: 1.3999999999999997
H: 12 / 42 : Minimum distance = 1.428428
 Setting new minimum interatomic distance after many failed placements: 1.3899999999999997
H: 15 / 42 : Minimum distance = 1.391036
 Setting new minimum interatomic distance after many failed placements: 1.3799999999999997
H: 17 / 42 : Minimum distance = 1.401930
 Setting new minimum interatomic distance after many failed placements: 1.3699999999999997
H: 21 / 42 : Minimum distance = 1.396905
 Setting new minimum interatomic distance after many failed placements: 1.3599999999999997
H: 24 / 42 : Minimum distance = 1.444709
 Setting new minimum interatomic distance after many failed placements: 1.3499999999999996
H: 30 / 42 : Minimum distance = 1.352800
 Setting new minimum interatomic distance after many failed placements: 1.3399999999999996
H: 31 / 42 : Minimum distance = 1.351173
 Setting new minimum interatomic di

C: 72 / 100 : Minimum distance = 1.831001
 Setting new minimum interatomic distance after many failed placements: 1.77
C: 75 / 100 : Minimum distance = 1.771143
 Setting new minimum interatomic distance after many failed placements: 1.76
C: 76 / 100 : Minimum distance = 1.762720
 Setting new minimum interatomic distance after many failed placements: 1.75
C: 79 / 100 : Minimum distance = 1.769986
 Setting new minimum interatomic distance after many failed placements: 1.74
C: 81 / 100 : Minimum distance = 1.760564
 Setting new minimum interatomic distance after many failed placements: 1.73
C: 82 / 100 : Minimum distance = 1.773728
 Setting new minimum interatomic distance after many failed placements: 1.72

 Setting new minimum interatomic distance after many failed placements: 1.71

 Setting new minimum interatomic distance after many failed placements: 1.7
C: 83 / 100 : Minimum distance = 1.711688
 Setting new minimum interatomic distance after many failed placements: 1.69
C: 85 / 100 

O: 19 / 25 : Minimum distance = 1.531707
 Setting new minimum interatomic distance after many failed placements: 1.4599999999999997
O: 21 / 25 : Minimum distance = 1.476803
 Setting new minimum interatomic distance after many failed placements: 1.4499999999999997
O: 22 / 25 : Minimum distance = 1.454081
 Setting new minimum interatomic distance after many failed placements: 1.4399999999999997
H: 3 / 50 : Minimum distance = 1.4559508
 Setting new minimum interatomic distance after many failed placements: 1.4299999999999997
H: 5 / 50 : Minimum distance = 1.472910
 Setting new minimum interatomic distance after many failed placements: 1.4199999999999997
H: 9 / 50 : Minimum distance = 1.487295
 Setting new minimum interatomic distance after many failed placements: 1.4099999999999997

 Setting new minimum interatomic distance after many failed placements: 1.3999999999999997
H: 12 / 50 : Minimum distance = 1.459765
 Setting new minimum interatomic distance after many failed placements: 1.389

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[['A' 'B' 'C']]


array([['A', '0', '1', '2'],
       ['B', '3', '4', '5'],
       ['C', '6', '7', '8']], dtype='<U21')